In [1]:
import cupy as cp
from pyqcu.cuda import define
from pyqcu.cuda import io
from pyqcu.cuda import qcu
from pyqcu.cuda.set import params, argv, set_ptrs
print('My rank is ', define.rank)
params[define._LAT_Y_] /= 2
params[define._LAT_XYZT_] /= 2
params[define._SET_PLAN_] = 2
params[define._VERBOSE_] = 1
params[define._PARITY_] = 0
gauge_filename = f"quda_wilson-clover-dslash-gauge_-{params[define._LAT_X_]}-{params[define._LAT_Y_]}-{params  [define._LAT_Z_]}-{params[define._LAT_T_]}-{params[define._LAT_XYZT_]}-{params[define._GRID_X_]}-{params[define._GRID_Y_]}-{params[define._GRID_Z_]}-{params[define._GRID_T_]}-{params[define._PARITY_]}-{params[define._NODE_RANK_]}-{params[define._NODE_SIZE_]}-{params[define._DAGGER_]}-f.h5"
print("Parameters:", params)
#############################
print("Gauge filename:", gauge_filename)
gauge = io.hdf5_xxxtzyx2grid_xxxtzyx(params, gauge_filename)
fermion_in_filename = gauge_filename.replace("gauge", "fermion-in")
print("Fermion in filename:", fermion_in_filename)
fermion_in = io.hdf5_xxxtzyx2grid_xxxtzyx(params, fermion_in_filename)
fermion_out_filename = gauge_filename.replace("gauge", "fermion-out")
print("Fermion out filename:", fermion_out_filename)
quda_fermion_out = io.hdf5_xxxtzyx2grid_xxxtzyx(params, fermion_out_filename)
#############################
fermion_out = cp.zeros_like(fermion_in)
print("Fermion out data:", fermion_out.data)
print("Fermion out shape:", fermion_out.shape)
#############################
qcu.applyInitQcu(set_ptrs, params, argv)
# qcu.applyWilsonDslashQcu(fermion_out, fermion_in, gauge, set_ptrs, params)
qcu.applyCloverDslashQcu(fermion_out, fermion_in, gauge, set_ptrs, params)
qcu.applyEndQcu(set_ptrs, params)
#############################
print("Fermion out data:", fermion_out.data)
print("Fermion out shape:", fermion_out.shape)
print("QUDA Fermion out data:", quda_fermion_out.data)
print("QUDA Fermion out shape:", quda_fermion_out.shape)
print("Difference:", cp.linalg.norm(fermion_out -
      quda_fermion_out)/cp.linalg.norm(quda_fermion_out))
#############################
io.grid_xxxtzyx2hdf5_xxxtzyx(fermion_out, params)
#############################
clover_ee = cp.zeros(shape=[define._LAT_S_, define._LAT_C_] +
                     list(fermion_in.shape), dtype=fermion_in.dtype)
clover_ee_inv = cp.zeros_like(clover_ee)
clover_oo = cp.zeros_like(clover_ee)
clover_oo_inv = cp.zeros_like(clover_ee)
#############################
params[define._VERBOSE_] = 1
params[define._PARITY_] = 0
params[define._SET_INDEX_] += 1
params[define._SET_PLAN_] = 2
#############################
qcu.applyInitQcu(set_ptrs, params, argv)
qcu.applyCloversQcu(clover_ee, clover_ee_inv, gauge, set_ptrs, params)
qcu.applyEndQcu(set_ptrs, params)
#############################
params[define._VERBOSE_] = 1
params[define._PARITY_] = 1
params[define._SET_INDEX_] += 1
params[define._SET_PLAN_] = 2
#############################
qcu.applyInitQcu(set_ptrs, params, argv)
qcu.applyCloversQcu(clover_oo, clover_oo_inv, gauge, set_ptrs, params)
qcu.applyEndQcu(set_ptrs, params)



    @@@@@@######QCU NOTES START######@@@@@@@
    Guide:
    0. Required: MPI(e.g. 4.1.2), CUDA(e.g. 12.4), CMAKE(e.g. 3.22.1), GCC(e.g. 11.4.0), HDF5-MPI(e.g. 1.10.7,'apt install libhdf5-mpi-dev && export HDF5_MPI="ON" && pip install --no-binary=h5py h5py').
    1. The libqcu.so was compiled when pyqcu setup in download_path/PyQCU/lib, please add this path to your LD_LIBRARY_PATH.
    2. The QCU(PyQCU) splite grid by x->y->z->t, lattice by x->y->z->t->p->d[x,y,z,t]->c->c or x->y->z->t->c->s(->p) and x->y->z->t->c->s->c->s(->p).
    3. The QUDA(PyQUDA) splite grid by t->z->y->x, lattice by c->c->x->y->z->t->p->d[x,y,z,t] or c->s->x->y->z->t(->p) and c->s->c->s->x->y->z->t(->p).
    4. The QCU input params in numpy array(dtype=np.int32), argv in  numpy array(dtype=np.float32 or float64) array, set_ptrs in numpy array(dtype=np.int64), other in cupy array(dtype=cp.complex64 or complex128).
    5. The smallest lattice size is (wilson:x=4,y=4,z=4,t=4;clover:x=8,y=8,z=8,t=8) that QCU support

In [2]:
_=cp.einsum("ab...,bc...->ac...",clover_ee.reshape(12,12,-1),clover_ee_inv.reshape(12,12,-1))
index=9
_[:,:,index][index][index]

array(1.0000002+1.436821e-09j, dtype=complex64)

In [3]:
#############################
params[define._VERBOSE_] = 1
params[define._PARITY_] = 0
params[define._SET_INDEX_] += 1
params[define._SET_PLAN_] = 1
#############################


In [4]:

qcu.applyInitQcu(set_ptrs, params, argv)


set_ptr:0x55f0de3e8550
long long set_ptr:94493009151312
gridDim.x               :16384
blockDim.x              :16
host_params[_LAT_X_]    :16
host_params[_LAT_Y_]    :16
host_params[_LAT_Z_]    :32
host_params[_LAT_T_]    :32
host_params[_LAT_XYZT_] :262144
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :0
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :0
host_params[_MAX_ITER_] :10000
host_params[_DATA_TYPE_]:3
host_params[_SET_INDEX_]:3
host_params[_SET_PLAN_] :1
host_params[_MG_X_]     :4
host_params[_MG_Y_]     :4
host_params[_MG_Z_]     :4
host_params[_MG_T_]     :8
host_params[_LAT_E_]    :24
host_params[_VERBOSE_]  :1
host_argv[_MASS_]       :-3.500000e+00
host_argv[_TOL_]        :1.000000e-09
host_argv[_SIGMA_]      :1.000000e-01
lat_2dim[_XY_]          :256
lat_2dim[_XZ_]          :512
lat_2dim[_XT_]          :512
lat_2dim[_YZ_]          :512
lat_2dim[_YT_]     

In [5]:

# qcu.applyCloverBistabCgQcu(fermion_out, fermion_in,
#                            gauge, clover_ee, clover_oo, clover_ee_inv, clover_oo_inv,  set_ptrs, params)


In [6]:

qcu.applyEndQcu(set_ptrs, params)
#############################

set_ptr:0x55f0de3e8550
long long set_ptr:94493009151312
gridDim.x               :16384
blockDim.x              :16
host_params[_LAT_X_]    :16
host_params[_LAT_Y_]    :16
host_params[_LAT_Z_]    :32
host_params[_LAT_T_]    :32
host_params[_LAT_XYZT_] :262144
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :0
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :0
host_params[_MAX_ITER_] :10000
host_params[_DATA_TYPE_]:3
host_params[_SET_INDEX_]:3
host_params[_SET_PLAN_] :1
host_params[_MG_X_]     :4
host_params[_MG_Y_]     :4
host_params[_MG_Z_]     :4
host_params[_MG_T_]     :8
host_params[_LAT_E_]    :24
host_params[_VERBOSE_]  :1
host_argv[_MASS_]       :-3.500000e+00
host_argv[_TOL_]        :1.000000e-09
host_argv[_SIGMA_]      :1.000000e-01
lat_2dim[_XY_]          :256
lat_2dim[_XZ_]          :512
lat_2dim[_XT_]          :512
lat_2dim[_YZ_]          :512
lat_2dim[_YT_]     

In [7]:
clover_ee.reshape(12,12,-1)

array([[[ 0.93711185+0.00000000e+00j,  0.967906  +0.00000000e+00j,
          1.0044008 +0.00000000e+00j, ...,  0.97643834+0.00000000e+00j,
          1.0749791 +0.00000000e+00j,  0.9636847 +0.00000000e+00j],
        [ 0.0123342 +2.95877978e-02j,  0.0885148 +8.78620073e-02j,
         -0.02502289+5.04544675e-02j, ...,  0.00949738-1.99944563e-02j,
         -0.0366797 +3.84190902e-02j, -0.0515602 -1.46666735e-01j],
        [-0.01463284-9.40655842e-02j,  0.02626852+9.28470790e-02j,
         -0.03774733+8.92679542e-02j, ...,  0.03719737+1.92588493e-02j,
         -0.18209937-1.31488703e-02j,  0.12748225+1.50117921e-02j],
        ...,
        [-0.        -0.00000000e+00j, -0.        -0.00000000e+00j,
         -0.        -0.00000000e+00j, ..., -0.        -0.00000000e+00j,
         -0.        -0.00000000e+00j, -0.        -0.00000000e+00j],
        [-0.        -0.00000000e+00j, -0.        -0.00000000e+00j,
         -0.        -0.00000000e+00j, ..., -0.        -0.00000000e+00j,
         -0.        